In [141]:
# Verificar se melhoria de resultados na divulgaçao performam melhor que o IBOVESPA
# Acessar mongo e pegar dados de divulgação por empresa
# verificar se houve melhoria de algum indicador específico
# se houve, verificar qual a rentabilidade da empresa até a próxima divulgação de resultados ou início da divulgação do terceiro trimestre
    # pegar os preços dos ativos no mysql

# _id: dia em string 27/07/2023
# _129283 = chave e o dicionario abaixo é o valor
# empresa
# "VALE S.A."
# nr_sequencial
# 129283
# descricao
# "Informações Trimestrais - 30/06/2023 - V1"
# data_doc
# "30/06/2023"
# data_envio
# "27/07/2023"
# hora_envio
# "27/07/2023 19:48:48"
# link_consulta
# "https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequen…"
# link_download
# "http://www.rad.cvm.gov.br/ENETCONSULTA/frmDownloadDocumento.aspx?Codig…"

In [142]:
class Stock:
    def __init__(self):
        self.empresa = ''
        self.codcvm = 0
        self.data_divulg = ''
        self.precoinicio = 0
        self.precofim = 0
        self.demfinanceiros= {}

    def inputvalores_primarios(self, id, cod_cvm, data_divulgacao):
        self.empresa = id
        self.codcvm = cod_cvm
        self.data_divulg = data_divulgacao

    def inputvalores_precos(self, preco_entrada, preco_saida):
        self.precoinicio = preco_entrada
        self.precofim = preco_saida

    def show_info(self):
        print(f'Nome empresa: {self.empresa}')
        print(f'Cod cvm: {self.codcvm}')
        print(f'Data divulgacao: {self.data_divulg}')
        print(f'Preco Inicio: {self.precoinicio}')
        print(f'Preco Fim: {self.precofim}')

import pandas as pd
import mysql.connector
# import datetime as datetime
import warnings
from datetime import datetime, date
warnings.filterwarnings('ignore')

db = mysql.connector.connect(
    user='cartor',
    password='Cartor1212',
    host='ddns.cartor.com.br',
    port='53306', # 43306
    database='console' # os.getenv('MYSQL_DATABASE'),
    )

def encontrar_close(df, stock_name, fdata_divulgacao):
    try:
        filtro = (df['Stock'] == stock_name) & (df['Date'] == fdata_divulgacao)
        close_value = df.loc[filtro, 'Close'].values[0]
        return close_value
    except IndexError:
        return None 
    
def converterstr_data(data_hora_string):
    data_hora = datetime.strptime(data_hora_string, "%d/%m/%Y %H:%M:%S")
    data = data_hora.date()
    return data    

msql = pd.read_sql_query("SELECT * from b3", db)
msql = msql.reset_index(drop=True)         

In [143]:
from pymongo import MongoClient
import re

# Acessando o banco MongoDB Compass
cliente = MongoClient(
    "ddns.cartor.com.br",
    port=57017,
    username='cartor',
    password='Cartor1212',
    authSource='admin',
    authMechanism='DEFAULT'
)

db = cliente.CVM
collection = 'Empresas_fin6'

def finddoc_byequity(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byid(db, collection_name, b3_code):
    count = 0
    while count != 1:
        collection = db[collection_name]
        documents = collection.find({'equity': b3_code})
        count = collection.count_documents({'equity': b3_code})
        if count > 1:
            print('Lista encontrada: ')
            print(type(documents))
            for doc in documents:
                print(doc['_id'])
            print('CODIGO EXISTENTE EM MAIS DE 01 DOCUMENTO')
            exit()
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

def finddoc_byname(db, collection_name, containing_word, trimestre=False):
    count = 0
    while count != 1:

        if trimestre:
            collection = db[collection_name]
            regex = re.compile(containing_word+trimestre, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})
        else:
            collection = db[collection_name]
            regex = re.compile(containing_word, re.IGNORECASE)
            documents = collection.find({'_id': regex})
            count = collection.count_documents({'_id': regex})

        if count > 1:
            print(type(documents))
            print('erro aconteceu - checar base')
            for doc in documents:
                print(doc['_id'])
            break
            # containing_word = input("Qual das empresas seria a solicitada: ")
        if count == 0:
            print("Não foi encontrada a empresa desejada.")
            return None  # Retorna None se não encontrar a empresa
    return documents

In [144]:
ROMI3 = Stock()
WEGE3 = Stock()
ASAI3 = Stock()


In [145]:
stock_list = ['ROMI3', 'WEGE3', 'ASAI3']
stock_info = [ ROMI3, WEGE3, ASAI3 ]

collection = 'divulgacaoinfo_resultados'

for i, stock in enumerate(stock_list):

    document = finddoc_byequity(db, 'Empresas_fin6', stock)
    df1 = document.next() # dicionario referente a stock pick

    id_empresa = df1["_id"]    
    # print(f'Acao: {stock}')
    # print(f'_id : {id_empresa}') 

    document = finddoc_byname(db, 'divulgacaoinfo_resultados', id_empresa, '_30/06/2023')
    df2 = document.next() # dicionario referente a stock pick
    id_divulgacao = df2["nome_empresa"]    

    Stock.inputvalores_primarios(stock_info[i], df1["_id"] , df2["cod_cvm"], df2["hora_envio"])

    # tratar data e pegar valores de preços
    preco_inicio = encontrar_close(msql, stock, converterstr_data(stock_info[i].data_divulg))     
    preco_fim = encontrar_close(msql, stock, converterstr_data('10/10/2023 00:00:00'))
    Stock.inputvalores_precos(stock_info[i], preco_inicio, preco_fim)

    # puxar o balanço
    


    Stock.show_info(stock_info[i])
    print(" ")

# for stock in stock_list:
#     document = finddoc_byequity(db, 'Empresas_fin6', stock)
#     df1 = document.next() # dicionario referente a stock pick

#     id_empresa = df1["_id"]    
#     print(f'Acao: {stock}')
#     print(f'_id : {id_empresa}\n') 

# document = finddoc_byequity(db, 'Empresas_fin6', stock_list[0])
# df1 = document.next() # dicionario referente a stock pick

# valor_cod_cvm = df1["_id"]
# print(valor_cod_cvm)

Nome empresa: ROMI S.A.
Cod cvm: 7510
Data divulgacao: 18/07/2023 18:08:09
Preco Inicio: 16.0
Preco Fim: 11.96
 
Nome empresa: WEG S.A.
Cod cvm: 5410
Data divulgacao: 19/07/2023 07:03:26
Preco Inicio: 37.91
Preco Fim: 35.3
 
Nome empresa: SENDAS DISTRIBUIDORA S.A.
Cod cvm: 25372
Data divulgacao: 26/07/2023 19:32:29
Preco Inicio: 12.87
Preco Fim: 12.11
 


In [146]:
db = cliente.CVM
collection = 'Empresas_fin6'

In [147]:
trimestres = []
for chave, valor in df1.items():
    if isinstance(chave, str):
        trimestres.append(chave)

In [148]:
dict_DFCons = df1['2023-06-30']
print(dict_DFCons.items())

for item in dict_DFCons.items():
    print(item)

dict_items([('DFConsolidado', {}), ('DFIndividual', {'DF Individual - Balanço Patrimonial Ativo': {'1': {'st_conta': 'True', 'ds_conta': 'Ativo Total', 'vl_conta': 40102000.0}, '1.01': {'st_conta': 'True', 'ds_conta': 'Ativo Circulante', 'vl_conta': 13058000.0}, '1.01.01': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.01': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.02': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.03': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.04': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.05': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.06': {'st_conta': 'True', 'ds_conta': 'Caixa e Equivalentes de Caixa', 'vl_conta': 4596000.0}, '1.01.01.07'

In [149]:
trimestres

['_id',
 'cod_cvm',
 'cnpj_cia',
 '2023-03-31',
 '2022-03-31',
 '2022-06-30',
 '2022-09-30',
 '2022-12-31',
 '2021-03-31',
 '2021-06-30',
 '2021-09-30',
 '2020-12-31',
 '2021-12-31',
 '2020-03-31',
 '2020-06-30',
 '2020-09-30',
 '2019-12-31',
 'b3_issuer_code',
 'equity',
 '2023-06-30',
 'QteTotalAcoes',
 '2023-09-30']

In [150]:
import pandas as pd
import mysql.connector
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')

db = mysql.connector.connect(
    user='cartor',
    password='Cartor1212',
    host='ddns.cartor.com.br',
    port='53306', # 43306
    database='console' # os.getenv('MYSQL_DATABASE'),
    )

def encontrar_close(df, stock, date):
    try:
        filtro = (df['Stock'] == stock) & (df['Date'] == date)
        close_value = df.loc[filtro, 'Close'].values[0]
        return close_value
    except IndexError:
        return None 

msql = pd.read_sql_query("SELECT * from b3", db)
msql = msql.reset_index(drop=True) 
print(encontrar_close(msql, 'DI1F24', datetime.date(2017, 1, 2)))    

11.42


In [151]:
datetime.date(2023, 10, 10)

datetime.date(2023, 10, 10)

In [152]:
msql.head()

,Date,Stock,Open,Close,Max,Min,Average,Volume
0,2017-01-02,DI1F24,11.51,11.42,11.54,11.41,NaN,NaN
1,2017-01-03,DI1F24,11.44,11.47,11.51,11.44,NaN,NaN
2,2017-01-04,DI1F24,11.56,11.58,11.58,11.56,NaN,NaN
3,2017-01-05,DI1F24,11.53,11.51,11.55,11.46,NaN,NaN
4,2017-01-06,DI1F24,11.47,11.54,11.54,11.47,NaN,NaN


In [153]:
valor = msql.loc[0, 'Date']
valor

datetime.date(2017, 1, 2)